In [9]:
import transformers
import os
from torch import nn


In [8]:
bert_model = transformers.BertTokenizer.from_pretrained(os.getenv('MODEL'))
last_hidde_state, pooled_output = bert_model(
    input_ids = encoding['input_ids'],
    attention_mask = encoding['attention_mask']
)

NameError: name 'encoding' is not defined

In [5]:
bert_model.config.hidden_size

PreTrainedTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

## Sentiment classifier

In [11]:
class SentimentClassifier(nn.Module):

    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(os.getenv('MODEL'))
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self):
        _, pooled_output = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask,
        )
        output = self.drop(pooled_output)
        output = self.out(output)

        return self.softmax(output)

In [12]:
model = SentimentClassifier(len(class_names))
model = model.to(os.getenv('GPU'))

NameError: name 'class_names' is not defined